<a href="https://colab.research.google.com/github/efearase/RL_with_sentiment/blob/main/features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from moexalgo import Ticker
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def calculate_RSI(data, period=14):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))

    return RSI

def calculate_MACD(data, short_period=12, long_period=26, signal_period=9):
    short_EMA = data.ewm(span=short_period, adjust=False).mean()
    long_EMA = data.ewm(span=long_period, adjust=False).mean()

    MACD = short_EMA - long_EMA
    signal_line = MACD.ewm(span=signal_period, adjust=False).mean()
    histogram = MACD - signal_line

    return MACD

In [3]:
names = pd.read_csv('Export_ru_securities-list_20240129.csv', encoding='windows-1251', delimiter=';', quotechar='"')
codes = names[((names['LIST_SECTION']=='Первый уровень')) & (names['SUPERTYPE']=='Акции')]['TRADE_CODE'].dropna()

df_list = []

for code in codes:
    data = Ticker(code).candles(date='2022-03-20', till_date='today', period='1D')

    data['RSI'] = calculate_RSI(data['close'])
    data['MACD']= calculate_MACD(data['close'])

    data['Symbols'] = code

    data.rename(columns={'close': 'Adj Close', 'high': 'High', 'low': 'Low', 'open': 'Open', 'volume': 'Volume', 'begin':'Date'}
                , inplace=True)

    df_list.append(data)

panel_data = pd.concat(df_list, ignore_index=True)

panel_data.sort_values(by='Date', inplace=True)
panel_data.drop(columns = ['value', 'end'], inplace=True)
panel_data.to_csv('stocks_data/stocks_panel_data.csv', index=False)

In [5]:
panel_data

,Open,Adj Close,High,Low,Volume,Date,RSI,MACD,Symbols
0,5.1,5.534,5.663,5.1,33721900.0,2022-03-24,NaN,0.000000,CBOM
25786,313.0,329.000,369.900,300.4,1767605.0,2022-03-24,NaN,0.000000,TATNP
24310,1130.0,1343.600,1449.000,1130.0,1330520.0,2022-03-24,NaN,0.000000,NVTK
1431,82.0,89.390,99.800,82.0,20195080.0,2022-03-24,NaN,0.000000,ALRS
17168,1315.0,1280.000,1439.000,1250.0,2674028.0,2022-03-24,NaN,0.000000,CHMF
...,...,...,...,...,...,...,...,...,...
8744,2290.0,2318.400,2325.600,2282.4,141898.0,2024-03-01,62.268804,32.266148,POSI
8254,980.0,982.600,997.800,963.8,1824866.0,2024-03-01,80.408591,62.533227,LSRG
24309,3723.0,3658.500,3726.000,3645.5,160829.0,2024-03-01,44.301075,-49.208009,SMLT
13725,5647.0,5648.000,5668.000,5632.0,21050.0,2024-03-01,48.351648,1.104029,BELU


In [ ]:
data